### Import Dependencies

In [ ]:
# for machine learning/neural network
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras_tuner as kt

# for data handling
import pandas as pd

# general use
from os.path import join

### Preprocessing

In [ ]:
# bring in the raw data
df0 = pd.read_csv(join("resources", "charity_data.csv"))

# preview the raw data
df0.head()

In [ ]:
# EIN and NAME are unnecessary for the neural net, so we'll drop them from the dataset
df1 = df0.drop(["EIN", "NAME", "STATUS"], axis = 1)

# preview the data
df1.head()

In [ ]:
# define unique item threshold
unique_item_count = 10

# check for columns that require modification
modify_columns = []
value_count_lists = []
for col in df1.nunique().items():
    if (col[1] > unique_item_count) and (df1[col[0]].dtype == "object"):
        modify_columns.append(col[0])
        value_count_lists.append(df1[col[0]].value_counts())
        print(df1[col[0]].value_counts())
        print()

In [ ]:
# specify cutoff values for the relevant columns
cutoff_values = [100, 10]

# modify the specified columns
for i in range(len(modify_columns)):
    
    # assemble a list of items to be replaced
    items_to_replace = []
    for item in value_count_lists[i].items():
        if item[1] < cutoff_values[i]:
            items_to_replace.append(item[0])
    
    # replace the items of the associated column
    for item in items_to_replace:
        df1[modify_columns[i]] = df1[modify_columns[i]].replace(item, "other")
    
    # display the modified column
    print(df1[modify_columns[i]].value_counts())
    print()

In [ ]:
# replace categorical data with numerical data
df2 = pd.get_dummies(df1)
df2.head()

In [ ]:
# define features and outputs
y = df2["IS_SUCCESSFUL"].values
X = df2.drop("IS_SUCCESSFUL", axis = 1).values

# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# create and fit the scaler
scaler = StandardScaler().fit(X_train)

# scale the features
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Preprocessing Function

In [ ]:
# perform the preprocessing steps
def prepare_data(df, removed_columns, cutoff_values, unique_item_count):
    df_1 = df.drop(removed_columns, axis = 1)
    
    # check for columns that require modification
    modify_columns = []
    value_count_lists = []
    for col in df_1.nunique().items():
        if (col[1] > unique_item_count) and (df_1[col[0]].dtype == "object"):
            modify_columns.append(col[0])
            value_count_lists.append(df_1[col[0]].value_counts())
    
    # modify the specified columns
    for i in range(len(modify_columns)):

        # assemble a list of items to be replaced
        items_to_replace = []
        for item in value_count_lists[i].items():
            if item[1] < cutoff_values[i]:
                items_to_replace.append(item[0])

        # replace the items of the associated column
        for item in items_to_replace:
            df_1[modify_columns[i]] = df_1[modify_columns[i]].replace(item, "other")

    # replace categorical data with numerical data
    df_2 = pd.get_dummies(df_1)

    # define features and outputs
    y = df_2["IS_SUCCESSFUL"].values
    X = df_2.drop("IS_SUCCESSFUL", axis = 1).values

    # split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

    # create and fit the scaler
    scaler = StandardScaler().fit(X_train)

    # scale the features
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return (X_train_scaled, X_test_scaled, y_train, y_test)

### Neural Net Builders

In [ ]:
# build the neural net model for quick iterations
def quick_build_model(X_train_scaled, X_test_scaled, y_train, y_test):
    
    # initialize the neural net model
    nn_model = tf.keras.models.Sequential()

    # define the input layer
    nn_model.add(tf.keras.layers.Dense(units = 80, activation = "relu", input_dim = X_train_scaled.shape[1]))

    # define the hidden layer(s)
    nn_model.add(tf.keras.layers.Dense(units = 30, activation = "relu"))

    # define the output layer
    nn_model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

    # show the model's summary
    nn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    # fit the model
    nn_model.fit(X_train_scaled, y_train, epochs = 20, verbose = 0)
    
    # return the model's evaluation
    return nn_model.evaluate(X_test_scaled, y_test, verbose = 0)

# build the neural net model for keras tuner
def tuner_build_model(hp):
    
    # instantiate the model
    model = tf.keras.models.Sequential()
    
    # populate activation function options
    activation_options = hp.Choice("activation", ["relu", "tanh", "sigmoid"])
    
    # populate initial layer neurons
    model.add(tf.keras.layers.Dense(
        units = hp.Int("first_units", 
                       min_value = 2, 
                       max_value = 6, 
                       step = 2), 
        activation = activation_options, 
        input_dim = X_train_scaled.shape[1]))
    
    # populate hidden layer neurons
    for i in range(hp.Int("num_layers", 1, 6)):
        model.add(tf.keras.layers.Dense(
            units = hp.Int("units_" + str(i),
                          min_value = 2,
                          max_value = 6,
                          step = 2),
            activation = activation_options))
    
    # populate output layer neurons
    model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))
    
    # compile the model
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    # return the compiled model
    return model

### Optimize with Iterative Functions

In [ ]:
columns_to_remove = [
    ["EIN", "NAME", "AFFILIATION"],
    ["EIN", "NAME", "USE_CASE"],
    ["EIN", "NAME", "ORGANIZATION"],
    ["EIN", "NAME", "STATUS"],
    ["EIN", "NAME", "INCOME_AMT"],
    ["EIN", "NAME", "SPECIAL_CONSIDERATIONS"],
    ["EIN", "NAME", "ASK_AMT"]]

cutoff_values_to_check = [
    [10, 10],
    [100, 10],
    [500, 10],
    [1000, 10],
    [10, 500],
    [100, 500],
    [500, 500],
    [1000, 500],
    [10, 1000],
    [100, 1000],
    [500, 1000],
    [1000, 1000],
    [10, 2000],
    [100, 2000],
    [500, 2000],
    [1000, 2000]]

columns = []
cv0 = []
cv1 = []
losses = []
accuracies = []
for removed_columns in columns_to_remove:
    for cutoff_values in cutoff_values_to_check:
        print(f"Columns: {removed_columns}")
        print(f"Values: {cutoff_values}")
        iX_train_scaled, iX_test_scaled, iy_train, iy_test = prepare_data(df0, removed_columns, cutoff_values, 10)
        model_loss, model_acc = quick_build_model(iX_train_scaled, iX_test_scaled, iy_train, iy_test)
        print(f"Loss: {model_loss:,.4f}, Accuracy: {model_acc:,.4f}")
        columns.append(removed_columns[-1])
        cv0.append(cutoff_values[0])
        cv1.append(cutoff_values[1])
        losses.append(model_loss)
        accuracies.append(model_acc)

In [ ]:
results_df = pd.DataFrame({
    "loss": losses,
    "acc": accuracies,
    "cv_0": cv0,
    "cv_1": cv1,
    "column": columns
})
results_df.head()

In [ ]:
results_df.loc[results_df["loss"] == results_df["loss"].min()]
# when loss is minimized...
# loss    -> 0.549294
# acc     -> 0.731195
# cv_0    -> 10
# cv_1    -> 10
# column  -> SPECIAL_CONSIDERATIONS

In [ ]:
results_df.loc[results_df["acc"] == results_df["acc"].max()]
# when accuracy is maximized
# loss    -> 0.550659
# acc     -> 0.733294
# cv_0    -> 100
# cv_1    -> 10
# column  -> STATUS

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(losses, accuracies)
plt.xlabel("loss")
plt.ylabel("acc")
plt.show()

If we remove the `STATUS` column and use cutoff values of 100 and 10 for `APPLICATION_TYPE` and `CLASSIFICATION` respectively, then we attain a little more accuracy and less loss than the original model.

### Compile, Train, and Evaluate with Keras Tuner

In [ ]:
# instantiate the tuner
tuner = kt.Hyperband(
    tuner_build_model,
    objective = "val_accuracy",
    max_epochs = 20,
    overwrite = True,
    hyperband_iterations = 2)

In [ ]:
# run the tuner
tuner.search(X_train_scaled, y_train, epochs = 20, validation_data = (X_test_scaled, y_test))

In [ ]:
# retrieve the highest performing hyperparameters
best_hps = tuner.get_best_hyperparameters(1)[0]

# preview the hyperparameters
best_hps.values

# activation -> sigmoid
# layer count -> 2
# neuron count for layer 1 -> 6
# neuron count for layer 2 -> 2

In [ ]:
# evaluate the model's performance
best_model = tuner.get_best_models(1)[0]
model_loss, model_acc = best_model.evaluate(X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss:,.4f}, Accuracy: {model_acc:,.4f}")

# OUTPUT...
# 268/268 - 0s - loss: 0.5727 - accuracy: 0.7341 - 375ms/epoch - 1ms/step
# Loss: 0.5702, Accuracy: 0.7350

In [ ]:
# save the model
best_model.save(join("output", "alphabet_soup_charity_optimized.h5"))